In [1]:
# %%capture
# AUTO_RECONNECT = True #@param {type:"boolean"}

# if AUTO_RECONNECT:
#   import IPython
#   from google.colab import output

#   display(IPython.display.Javascript('''
#   function ClickConnect(){
#     btn = document.querySelector("colab-connect-button")
#     if (btn != null){
#       console.log("Click colab-connect-button"); 
#       btn.click() 
#       }
    
#     btn = document.getElementById('ok')
#     if (btn != null){
#       console.log("Click reconnect"); 
#       btn.click() 
#       }
#     }
    
#   setInterval(ClickConnect,60000)
#   '''))

In [2]:
## Standard libraries
import os
import json
import math
import numpy as np 
import time

## Imports for plotting
import matplotlib.pyplot as plt
%matplotlib inline 
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('svg', 'pdf') # For export
from matplotlib.colors import to_rgb
import matplotlib
matplotlib.rcParams['lines.linewidth'] = 2.0
import seaborn as sns
sns.reset_orig()
sns.set()

## Progress bar
from tqdm.notebook import tqdm

## PyTorch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data
import torch.optim as optim
# Torchvision
import torchvision
from torchvision.datasets import CIFAR10
from torchvision import transforms
# PyTorch Lightning
try:
    import pytorch_lightning as pl
except ModuleNotFoundError: # Google Colab does not have PyTorch Lightning installed by default. Hence, we do it here if necessary
    !pip install --quiet pytorch-lightning>=1.4
    import pytorch_lightning as pl
from pytorch_lightning.callbacks import LearningRateMonitor, ModelCheckpoint

# Path to the folder where the datasets are/should be downloaded (e.g. CIFAR10)
DATASET_PATH = "../data"
# Path to the folder where the pretrained models are saved
CHECKPOINT_PATH = "../saved_models/tutorial7"

# Setting the seed
pl.seed_everything(42)

# Ensure that all operations are deterministic on GPU (if used) for reproducibility
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
print(device)

/home/pc/anaconda3/envs/tor/lib/python3.7/site-packages/ipykernel_launcher.py:12: DeprecationWarning: `set_matplotlib_formats` is deprecated since IPython 7.23, directly use `matplotlib_inline.backend_inline.set_matplotlib_formats()`
  if sys.path[0] == "":
Global seed set to 42


cuda:0


In [3]:
# import urllib.request
# from urllib.error import HTTPError
# # Github URL where saved models are stored for this tutorial
# base_url = "https://raw.githubusercontent.com/phlippe/saved_models/main/tutorial7/"
# # Files to download
# pretrained_files = ["NodeLevelMLP.ckpt", "NodeLevelGNN.ckpt", "GraphLevelGraphConv.ckpt"]

# # Create checkpoint path if it doesn't exist yet
# os.makedirs(CHECKPOINT_PATH, exist_ok=True)

# # For each file, check whether it already exists. If not, try downloading it.
# for file_name in pretrained_files:
#     file_path = os.path.join(CHECKPOINT_PATH, file_name)
#     if "/" in file_name:
#         os.makedirs(file_path.rsplit("/",1)[0], exist_ok=True)
#     if not os.path.isfile(file_path):
#         file_url = base_url + file_name
#         print(f"Downloading {file_url}...")
#         try:
#             urllib.request.urlretrieve(file_url, file_path)
#         except HTTPError as e:
#             print("Something went wrong. Please try to download the file from the GDrive folder, or contact the author with the full output including the following error:\n", e)

In [3]:
# torch geometric
try: 
    import torch_geometric
except ModuleNotFoundError:
    # Installing torch geometric packages with specific CUDA+PyTorch version. 
    # See https://pytorch-geometric.readthedocs.io/en/latest/notes/installation.html for details 
    TORCH = torch.__version__.split('+')[0]
    CUDA = 'cu' + torch.version.cuda.replace('.','')

    !pip install torch-scatter     -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
    !pip install torch-sparse      -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
    !pip install torch-cluster     -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
    !pip install torch-spline-conv -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
    !pip install torch-geometric 
    import torch_geometric
import torch_geometric.nn as geom_nn
import torch_geometric.data as geom_data

Looking in links: https://pytorch-geometric.com/whl/torch-1.13.1+cu117.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 832.5 kB/s eta 0:00:00m eta 0:00:010:00:01
Looking in links: https://pytorch-geometric.com/whl/torch-1.13.1+cu117.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 849.7 kB/s eta 0:00:00m eta 0:00:010:00:01
Looking in links: https://pytorch-geometric.com/whl/torch-1.13.1+cu117.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 849.0 kB/s eta 0:00:00m eta 0:00:010:00:01
Looking in links: https://pytorch-geometric.com/whl/torch-1.13.1+cu117.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 872.9/872.9 kB 489.0 kB/s eta 0:00:000:00:010:00:01:01
  Using cached torch_geometric-2.2.0.tar.gz (564 kB)
  Preparing metadata (setup.py) ... done
  Created wheel for torch-geometric: filename=torch_geometric-2.2.0-py3-none-any.whl size=773285 sha256=0633bb5f70c5574f404b6046f94bf9a8393776f21b32fca538758ddeb53accca
  Stored in directory: /home/pc/.c

In [4]:
!pip install --upgrade --no-cache-dir gdown

In [6]:
!gdown 1f30APlPxmKuQHJW6xRFP1g7RZwc0f8nf

Downloading...
From: https://drive.google.com/uc?id=1f30APlPxmKuQHJW6xRFP1g7RZwc0f8nf
To: /content/graph_data.zip
100% 184M/184M [00:04<00:00, 38.8MB/s]


In [7]:
!unzip graph_data.zip

Archive:  graph_data.zip
  inflating: graph_data              


In [6]:
import pickle

In [8]:
data = pickle.load(open('./data/graph_data', 'rb'))

In [9]:
import pandas as pd
import random

In [10]:
# sorted(data.items(), key=lambda x: random.random())[:5]

In [11]:
len(data['دوچرخه'])

1815066

In [12]:
len(data)

270099

In [ ]:
import numpy as np
import networkx as nx
from gensim.models import Word2Vec
import torch
from torch_geometric.data import Data
from torch_geometric.nn import Node2Vec

In [ ]:
queries = list(data.keys())
products = list(set([product for query in data.values() for product in query]))
query2id = {query: i for i, query in enumerate(queries)}
product2id = {product: i for i, product in enumerate(products)}

In [ ]:
edges = []
for i, q1 in enumerate(queries):
    for j, q2 in enumerate(queries):
        if i != j:
            common_products = set(data[q1]).intersection(set(data[q2]))
            if len(common_products) > 50:
                edges.append((i, j, len(common_products)))


In [ ]:
G = nx.Graph()
G.add_nodes_from(queries)
G.add_weighted_edges_from(edges)
adj_matrix = np.zeros((len(queries), len(queries)))
for i, query_i in enumerate(queries):
    for j, query_j in enumerate(queries):
        if G.has_edge(query_i, query_j):
            adj_matrix[i][j] = G[query_i][query_j]['weight']

In [ ]:
print(G)

In [ ]:
# Step 3: Node2Vec Embedding
edge_index = torch.tensor(np.array(G.edges()).T, dtype=torch.long)
x = torch.tensor(np.eye(len(queries)), dtype=torch.float)
data = Data(x=x, edge_index=edge_index)
model = Node2Vec(data.edge_index, embedding_dim=128, walk_length=10,
                 context_size=5, walks_per_node=10, num_negative_samples=1,
                 p=1, q=1, sparse=True)
loader = model.loader(batch_size=128, shuffle=True, num_workers=4)
optimizer = torch.optim.SparseAdam(list(model.parameters()), lr=0.01)


In [ ]:
# Get embeddings of queries
model.train()
for epoch in range(10):
    for pos_rw, neg_rw in loader:
        optimizer.zero_grad()
        loss = model.loss(pos_rw.to(device), neg_rw.to(device))
        loss.backward()
        optimizer.step()
    model.eval()
    z = model()
    model.train()

query_embeddings = z.detach().numpy()[:len(queries)]

In [ ]:
query_embeddings.shape

In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

# Cluster the queries using KMeans
kmeans = KMeans(n_clusters=3, random_state=0).fit(query_embeddings)

# Evaluate the clustering performance
silhouette_avg = silhouette_score(query_embeddings, kmeans.labels_)
print(f"Silhouette Score: {silhouette_avg}")


In [ ]:
# save node edge and graph into drive
import pickle

with open('edges.pkl', 'wb') as f:
    pickle.dump(edges, f)

with open('query2id.pkl', 'wb') as f:
    pickle.dump(query2id, f)

with open('product2id.pkl', 'wb') as f:
    pickle.dump(product2id, f)

with open('queries.pkl', 'wb') as f:
    pickle.dump(queries, f)

with open('products.pkl', 'wb') as f:
    pickle.dump(products, f)

In [ ]:
# save node edge and graph into drive

with open('adj_matrix', 'wb') as f:
    pickle.dump(adj_matrix, f)

with open('G.pkl', 'wb') as f:
    pickle.dump(G, f)
    

In [13]:
import numpy as np
import networkx as nx
import torch
from torch_geometric.data import Data
from torch_geometric.nn import Node2Vec
from scipy.sparse import lil_matrix

In [14]:
queries = set(data.keys())
products = set(product for query in data.values() for product in query)
query2id = {query: i for i, query in enumerate(queries)}
product2id = {product: i for i, product in enumerate(products)}

In [15]:
# Compute the product sets for each query
query_products = {query2id[query]: set([product2id[product] for product in data[query]]) for query in queries}

In [16]:
# Compute the intersection of product sets for pairs of queries
edges = []
for i, q1 in enumerate(queries):
    for j, q2 in enumerate(queries):
        if i < j:
            common_products = query_products[i].intersection(query_products[j])
            if len(common_products) > 50:
                edges.append((i, j, len(common_products)))

In [ ]:
# Create the graph and adjacency matrix
G = nx.Graph()
G.add_nodes_from(queries)
G.add_weighted_edges_from(edges)
adj_matrix = lil_matrix((len(queries), len(queries)))
for i, query_i in enumerate(queries):
    for j, query_j in enumerate(queries):
        if i < j and G.has_edge(query_i, query_j):
            adj_matrix[i, j] = G[query_i][query_j]['weight']

In [ ]:
# Convert adjacency matrix to PyTorch sparse tensor
edge_index = torch.tensor(adj_matrix.nonzero(), dtype=torch.long).t().contiguous()
edge_weight = torch.tensor(adj_matrix.data, dtype=torch.float)
data = Data(edge_index=edge_index, edge_attr=edge_weight, num_nodes=len(queries))

In [ ]:
# Use PyTorch sparse tensor instead of NumPy array for x
x = torch.eye(len(queries)).to_sparse()


In [ ]:
# Train Node2Vec model
model = Node2Vec(data.edge_index, embedding_dim=128, walk_length=10,
                 context_size=5, walks_per_node=10, num_negative_samples=1,
                 p=1, q=1, sparse=True)
loader = model.loader(batch_size=128, shuffle=True, num_workers=4)
optimizer = torch.optim.SparseAdam(list(model.parameters()), lr=0.01)


In [ ]:
model.train()
for epoch in range(10):
    for pos_rw, neg_rw in loader:
        optimizer.zero_grad()
        loss = model.loss(pos_rw.to(device), neg_rw.to(device))
        loss.backward()
        optimizer.step()
    model.eval()
    z = model()
    model.train()


In [ ]:
query_embeddings = z.detach().cpu().numpy()[:len(queries)]


In [ ]:
# Cluster the query embeddings
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

kmeans = KMeans(n_clusters=3, random_state=0).fit(query_embeddings)
silhouette_avg = silhouette_score(query_embeddings, kmeans.labels_)
print(f"Silhouette Score: {silhouette_avg}")
